In [ ]:
from pathlib import Path
import numpy as np
from scipy import constants as C
from mmwave.rma import rma, echo_plot
from mmwave.util import load_frame

In [ ]:
output_dir = Path("/home/tomin/python/mmwave_postproc/outdoor_20241121_152633")
output_dir = Path("/home/tomin/python/mmwave_postproc/outdoor_20241121_143316")

frame_file, cfg = load_frame(output_dir)

num_sample = cfg.mimo.profile.numAdcSamples
adcStartTime = cfg.mimo.profile.adcStartTime  # us
startFrequency = cfg.mimo.profile.startFrequency * 1e9
K = cfg.mimo.profile.frequencySlope * 1e12  # Slope const (hz/s)
Fs = cfg.mimo.profile.adcSamplingFrequency * 1e3  # Sampling rate (sps)

F0 = startFrequency + adcStartTime * K * 1e-6 + num_sample // 2 / Fs * K  # Center frequency

dx = cfg.bracket.profile.dx
dy = cfg.bracket.profile.dy  # Sampling distance at x (horizontal) y (vertical) axis in mm
row = cfg.bracket.profile.row
col = cfg.bracket.profile.col

c = C.c
Ts = 1 / Fs  # Sampling period
k = 2 * np.pi * F0 / c  # Wave number
print(f"k:{k}")

In [ ]:
tx_idx = 1
rx_idx = 1
ID_select = 21
nFFTtime = num_sample  # Number of FFT points for Spatial-FFT
tI = 183  # mm
R = c / 2 * (ID_select / (K * Ts * nFFTtime)) - tI / 1000

Echo = frame_file[:, :, tx_idx, rx_idx, :, 0] + 1j * frame_file[:, :, tx_idx, rx_idx, :, 1]
Sr: np.ndarray = np.fft.fft(Echo)
Sr = Sr[:, :, ID_select - 1]
Sr[Sr == 0] = 1e-10
echo_plot(Sr, "source", dx, dy)

In [ ]:
reconstructed_image = rma(Sr, dx, dy, R, k)
echo_plot(reconstructed_image, "reconstructed_image", dx, dy, rma=True)